In [36]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ray import tune
import torch
import os
%matplotlib inline

In [39]:
all_pristine = [cv2.imread('images/'+i) for i in sorted(os.listdir('images'))]
all_labels = [cv2.imread('labels/'+i) for i in sorted(os.listdir('labels'))]

In [37]:
def run_hough(test_pristine,show=False,return_build_mask=False,\
              blur=9,dp=1,param1=50,param2=20):
    #HOUGH_GRADIENT, HOUGH_GRADIENT_ALT eventually? Same with Gaussian vs. Median
    #blur: in list [3,5,7,9,11,13,15,17]
    #dp: float in range 1 to 3
    #param1: int in range(10,1000)
    #param2: int in range(5,100)
    gray = cv2.medianBlur(cv2.cvtColor(test_pristine,cv2.COLOR_RGB2GRAY), blur)
    circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,dp,20,
                                param1=param1,param2=param2,minRadius=0,maxRadius=60)
    cimg = test_pristine.copy()
    circles = np.uint16(np.around(circles))
    if show:
        for i in circles[0,:]:
            cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
        plt.figure(figsize=(16,8))
        plt.imshow(cimg)
        plt.show()
    out_dict = {}
    all_boxes = []
    scores = []
    labels = []
    for i in range(len(circles[0,:])):
        all_boxes.append([circles[0,:][i][0]-circles[0,:][i][2],\
                          circles[0,:][i][1]-circles[0,:][i][2],\
                          circles[0,:][i][0]+circles[0,:][i][2],\
                          circles[0,:][i][1]+circles[0,:][i][2]])
        scores.append(1./(i+1))
        labels.append(1)
    out_dict['boxes']=torch.Tensor(all_boxes)
    out_dict['scores']=torch.Tensor(scores)
    out_dict['labels']=torch.Tensor(labels)
    if return_build_mask:
        build_mask = np.zeros(test_pristine.shape)
        for i in circles[0,:]:
            cv2.circle(build_mask,(i[0],i[1]),i[2],(255,255,255),-1)
        return build_mask, out_dict
    return out_dict

In [34]:
#Quick test
out_dict = run_hough(all_pristine[0])

In [ ]:
def training_function(config):
    # Hyperparameters
    blur, dp, param1, param2 = config['blur'], config['dp'], \
                                config['param1'], config['param2']
    mAP_loss_li = []
    for test_pristine in all_pristine:
        hough_dict = run_hough(test_pristine,blur=blur,\
                               dp=dp,param1=param1,param2=param2)
        #DEFINE GT_DICT FOR CORRESPONDING IMAGE. DEFINE mAP_loss
        mAP_loss_li.append(mean_loss=mAP(hough_dict,GT_dict))
    tune.report(mean_loss=np.average(mAP_loss_li))


analysis = tune.run(
    training_function,
    config={
        "blur": tune.choice([3,5,7,9,11,13,15,17]),
        "dp": tune.grid_search([1,3]),
        "param1": tune.choice(range(10,1000)),
        "param2": tune.choice(range(5,100))
    })

print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))

# Get a dataframe for analyzing trial results.
df = analysis.results_df